Required Imports

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception

Getting the OCTMNIST dataset from PIP

In [ ]:
!pip install medmnist
from medmnist import OCTMNIST
dataset_train = OCTMNIST(split="train", download=True)
dataset_test = OCTMNIST(split="test", download=True)
dataset_validate = OCTMNIST(split="val", download = True)

Shape of the dataset-images

In [ ]:
x_train, y_train = dataset_train.imgs, dataset_train.labels
x_test, y_test = dataset_test.imgs, dataset_test.labels
x_val, y_val = dataset_validate.imgs, dataset_validate.labels

print(x_train.shape, y_train.shape, x_train[0].shape)

(97477, 28, 28) (97477, 1) (28, 28)


Preprocessing the images - Resize and 3-channel conversion

In [ ]:
import cv2
import numpy as np

def preprocess_images(images):
    preprocessed_images = []
    for image in images:
        resized_image = cv2.resize(image, (92, 92))
        rgb_image = cv2.cvtColor(resized_image, cv2.COLOR_GRAY2RGB)
        preprocessed_images.append(rgb_image)
    return np.array(preprocessed_images)

In [ ]:
resized_x_train = preprocess_images(x_train)
resized_x_test = preprocess_images(x_test)
resized_x_val = preprocess_images(x_val)
resized_x_train.shape, resized_x_test.shape, resized_x_val.shape

((97477, 92, 92, 3), (1000, 92, 92, 3), (10832, 92, 92, 3))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the Models

In [ ]:
#The correct path to the trained DenseNet121 model has to be provided here
model1 = tf.keras.models.load_model('/content/drive/MyDrive/DenseNet121_92_76.8%.h5')

In [ ]:
#The correct path to the trained InceptionV3 model has to be provided here
model2 = tf.keras.models.load_model('/content/drive/MyDrive/OCTMNIST__inceptionv3.h5')

In [ ]:
num_samples = len(resized_x_test)
num_classes = 4

In [ ]:
print(num_samples)

1000


In [ ]:
predictions = np.zeros((len(resized_x_test), num_classes))

model_predictions1 = model1.predict(resized_x_test)
predictions += model_predictions1
model_predictions2 = model2.predict(resized_x_test)
predictions += model_predictions2

32/32 [==============================] - 0s 14ms/step


In [ ]:
model_predictions1.shape

(1000, 4)

Using Product Rule Ensemble technique

In [ ]:
ensemble_predictions_product_rule = np.multiply(model_predictions1, model_predictions2)

In [ ]:
ensemble_predictions_pr = np.argmax(ensemble_predictions_product_rule, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
y_true = y_test
accuracy = accuracy_score(y_true, ensemble_predictions_pr)
print("Ensemble Accuracy:", accuracy)

Ensemble Accuracy: 0.782


In [ ]:
from sklearn.metrics import classification_report

classification_report_pr = classification_report(y_true, ensemble_predictions_pr)

print(classification_report_pr)

              precision    recall  f1-score   support

           0       0.65      0.97      0.78       250
           1       0.96      0.86      0.91       250
           2       0.98      0.32      0.49       250
           3       0.76      0.97      0.86       250

    accuracy                           0.78      1000
   macro avg       0.84      0.78      0.76      1000
weighted avg       0.84      0.78      0.76      1000

